In [1]:
import cv2
import numpy as np
from cvzone.HandTrackingModule import HandDetector
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

In [2]:
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)
model = tf.keras.models.load_model('keras_model.h5')

In [3]:
class_names = ["ok", "rock", "silent", "thanks"]

def preprocess_hand(hand_img):
    h, w, _ = hand_img.shape
    aspect_ratio = w / h
    size = (224, 224)
    
    # Maintain aspect ratio with padding
    if aspect_ratio > 1:
        new_w = size[0]
        new_h = int(new_w / aspect_ratio)
    else:
        new_h = size[1]
        new_w = int(new_h * aspect_ratio)
    
    resized = cv2.resize(hand_img, (new_w, new_h))
    padded = np.full((size[1], size[0], 3), (0, 0, 0), dtype=np.uint8)
    
    # Center the hand image
    x_offset = (size[0] - new_w) // 2
    y_offset = (size[1] - new_h) // 2
    padded[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = resized
    
    grayscale = cv2.cvtColor(padded, cv2.COLOR_BGR2GRAY)
    
    return cv2.cvtColor(grayscale, cv2.COLOR_GRAY2BGR)

while True:
    success, img = cap.read()
    if not success:
        continue
    
    hands, _ = detector.findHands(img, draw=False)

    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']
        
        hand_img = img[max(0, y-10):min(img.shape[0], y+h+10),
                      max(0, x-10):min(img.shape[1], x+w+10)]
        
        if hand_img.size == 0:
            continue

        # Apply preprocessing
        processed = preprocess_hand(hand_img)
        

        input_image = np.expand_dims(processed, axis=0).astype(np.float32)
        input_image /= 255.0
        
        # Make prediction
        predictions = model.predict(input_image)
        class_index = np.argmax(predictions[0])
        confidence = predictions[0][class_index]
        
        # Display results
        cv2.rectangle(img, (x-10, y-10), (x+w+10, y+h+10), (0, 255, 0), 2)
        cv2.putText(img, f"{class_names[class_index]} {confidence:.2f}",
                    (x, y-30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Gesture Recognition", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 22ms/step
